In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

Target:

    Add Data augmentation to improve accuracy

Result:

    1st run:
      Change : Added GAP layer with kernel size = 6
      Parameters : 8,838
      Train accuracy = 98.71%
      Test accuracy = 99.21%%

    2nd run : 
      Change : Batch normalization to every layer
      Parameters : 8,954
      Test accuracy : very low - 30%
 

Analysis:

    1st iteration - No overfitting, we can improve the model
    2nd iteration - Adding batch norm to all layers and removing dropout from 6th conv layer has reduced the accuracy drastically.
    


In [0]:
# Original network with less parameters, so that training is also faster
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drop1 = nn.Dropout(0.1)                        #                       GRF - 1   J - 1
        self.conv1 = nn.Conv2d(1, 16, 3, padding=0)         #input - 28 OUtput - 26  J - 1  GRF - 3
        self.conv2 = nn.Conv2d(16, 10, 3, padding=0)        #input - 26 OUtput - 24  J - 1  GRF - 5 
        self.pool1 = nn.MaxPool2d(2, 2)                     #input - 24 OUtput - 12  J - 2  GRF - 7
        self.bnm2d1 = nn.BatchNorm2d(10) 
        self.conv3 = nn.Conv2d(10, 10, 3, padding=0)        #input - 12 OUtput - 10  J - 2  GRF - 11
        self.conv4 = nn.Conv2d(10, 16, 3, padding=0)        #input - 10 OUtput - 8   J - 2  GRF - 15
        # self.pool2 = nn.MaxPool2d(2, 2)
        self.bnm2d2 = nn.BatchNorm2d(16) 
        self.conv5 = nn.Conv2d(16, 16, 3, padding=0)        #input - 8  OUtput - 6   J - 2  GRF - 19
        self.conv6 = nn.Conv2d(16, 16, 3, padding=1)        #input - 6  OUtput - 6   J - 2  GRF - 23
        # self.conv7 = nn.Conv2d(16, 16, 3, padding=1)      #input - 6  OUtput - 6 
        self.gap = nn.AvgPool2d(kernel_size=6)
        self.conv8 = nn.Conv2d(16,10,1)                     

    def forward(self, x):
        x = self.bnm2d1(self.pool1(F.relu(self.drop1(self.conv2(F.relu(self.drop1(self.conv1(x))))))))
        x = self.bnm2d2(F.relu(self.drop1(self.conv4(F.relu(self.drop1(self.conv3(x)))))))
        x = self.conv6(F.relu(self.drop1(self.conv5(x))))
        # x = self.conv7(x)
        x = self.gap(x)
        x = self.conv8(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)
        

In [21]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
           Dropout-2           [-1, 16, 26, 26]               0
            Conv2d-3           [-1, 10, 24, 24]           1,450
           Dropout-4           [-1, 10, 24, 24]               0
         MaxPool2d-5           [-1, 10, 12, 12]               0
       BatchNorm2d-6           [-1, 10, 12, 12]              20
            Conv2d-7           [-1, 10, 10, 10]             910
           Dropout-8           [-1, 10, 10, 10]               0
            Conv2d-9             [-1, 16, 8, 8]           1,456
          Dropout-10             [-1, 16, 8, 8]               0
      BatchNorm2d-11             [-1, 16, 8, 8]              32
           Conv2d-12             [-1, 16, 6, 6]           2,320
          Dropout-13             [-1, 16, 6, 6]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-0.7,0.7), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
train_acc = []
test_acc = []
train_losses = []
test_losses = []

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0

    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        y_pred = model(data)

        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        pred = y_pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        # pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
# def __init__(self):
#     super(Net, self).__init__()
#     self.drop1 = nn.Dropout(0.1)  
#     self.conv1 = nn.Conv2d(1, 16, 3, padding=0)         #input - 28 OUtput - 26 RF
#     self.conv2 = nn.Conv2d(16, 10, 3, padding=0)        #input - 26 OUtput - 24
#     self.pool1 = nn.MaxPool2d(2, 2)                     #input - 24 OUtput - 12
#     self.bnm2d1 = nn.BatchNorm2d(10) 
#     self.conv3 = nn.Conv2d(10, 10, 3, padding=0)        #input - 12 OUtput - 10
#     self.conv4 = nn.Conv2d(10, 16, 3, padding=0)        #input - 10 OUtput - 8
#     # self.pool2 = nn.MaxPool2d(2, 2)
#     self.bnm2d2 = nn.BatchNorm2d(16) 
#     self.conv5 = nn.Conv2d(16, 16, 3, padding=0)        #input - 8  OUtput - 6
#     self.conv6 = nn.Conv2d(16, 16, 3, padding=1)        #input - 6  OUtput - 6
#     # self.conv7 = nn.Conv2d(16, 16, 3, padding=1)      #input - 6  OUtput - 6
#     self.gap = nn.AvgPool2d(kernel_size=6)
#     self.conv8 = nn.Conv2d(16,10,1)                     

# def forward(self, x):
#     x = self.bnm2d1(self.pool1(F.relu(self.drop1(self.conv2(F.relu(self.drop1(self.conv1(x))))))))
#     x = self.bnm2d2(F.relu(self.drop1(self.conv4(F.relu(self.drop1(self.conv3(x)))))))
#     x = self.conv6(F.relu(self.drop1(self.conv5(x))))
#     # x = self.conv7(x)
#     x = self.gap(x)
#     x = self.conv8(x)
#     x = x.view(-1, 10)
#     return F.log_softmax(x)


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.3312416672706604 batch_id=468 Accuracy=72.97: 100%|██████████| 469/469 [00:24<00:00, 18.82it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1247, Accuracy: 9638/10000 (96.38%)



loss=0.07011356204748154 batch_id=468 Accuracy=96.13: 100%|██████████| 469/469 [00:24<00:00, 19.06it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0767, Accuracy: 9760/10000 (97.60%)



loss=0.09891402721405029 batch_id=468 Accuracy=97.37: 100%|██████████| 469/469 [00:25<00:00, 18.72it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0242, Accuracy: 9921/10000 (99.21%)



loss=0.039024677127599716 batch_id=468 Accuracy=98.79: 100%|██████████| 469/469 [00:25<00:00, 18.75it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0271, Accuracy: 9916/10000 (99.16%)



loss=0.02889920212328434 batch_id=468 Accuracy=98.85: 100%|██████████| 469/469 [00:24<00:00, 18.79it/s]



Test set: Average loss: 0.0319, Accuracy: 9901/10000 (99.01%)



In [19]:
# def __init__(self):
#     super(Net, self).__init__()
#     self.drop1 = nn.Dropout(0.1)  
#     self.conv1 = nn.Conv2d(1, 16, 3, padding=0)         #input - 28 OUtput - 26 RF
#     self.conv2 = nn.Conv2d(16, 10, 3, padding=0)        #input - 26 OUtput - 24
#     self.pool1 = nn.MaxPool2d(2, 2)                     #input - 24 OUtput - 12
#     self.bnm2d1 = nn.BatchNorm2d(10) 
#     self.conv3 = nn.Conv2d(10, 10, 3, padding=0)        #input - 12 OUtput - 10
#     self.conv4 = nn.Conv2d(10, 16, 3, padding=0)        #input - 10 OUtput - 8
#     # self.pool2 = nn.MaxPool2d(2, 2)
#     self.bnm2d2 = nn.BatchNorm2d(16) 
#     self.conv5 = nn.Conv2d(16, 16, 3, padding=0)        #input - 8  OUtput - 6
#     self.conv6 = nn.Conv2d(16, 16, 3, padding=1)        #input - 6  OUtput - 6
#     # self.conv7 = nn.Conv2d(16, 16, 3, padding=1)      #input - 6  OUtput - 6
#     self.gap = nn.AvgPool2d(kernel_size=6)
#     self.conv8 = nn.Conv2d(16,10,1)                     

# def forward(self, x):
#     x = self.pool1(self.drop1(F.relu(self.bnm2d1(self.conv2(self.drop1(F.relu(self.bnm2d2(self.conv1(x)))))))))
#     x = self.drop1(F.relu(self.bnm2d2(self.conv4(self.drop1(F.relu(self.bnm2d1(self.conv3(x))))))))
#     x = self.bnm2d2(self.conv6(F.relu(self.bnm2d2(self.conv5(x)))))
#     # x = self.conv7(x)
#     x = self.gap(x)
#     x = self.conv8(x)
#     x = x.view(-1, 10)
#     return F.log_softmax(x)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.17681317031383514 batch_id=468 Accuracy=87.38: 100%|██████████| 469/469 [00:25<00:00, 18.43it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 2.0335, Accuracy: 3809/10000 (38.09%)



loss=0.043225664645433426 batch_id=468 Accuracy=97.52: 100%|██████████| 469/469 [00:25<00:00, 18.52it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 1.9129, Accuracy: 3630/10000 (36.30%)



loss=0.09559369832277298 batch_id=468 Accuracy=98.08: 100%|██████████| 469/469 [00:25<00:00, 18.40it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 2.5744, Accuracy: 2295/10000 (22.95%)



loss=0.03138168528676033 batch_id=468 Accuracy=99.03: 100%|██████████| 469/469 [00:25<00:00, 18.30it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 3.5937, Accuracy: 1328/10000 (13.28%)



loss=0.032281603664159775 batch_id=468 Accuracy=99.03: 100%|██████████| 469/469 [00:25<00:00, 18.51it/s]



Test set: Average loss: 3.1174, Accuracy: 1960/10000 (19.60%)



In [24]:
# def __init__(self):
#     super(Net, self).__init__()
#     self.drop1 = nn.Dropout(0.1)  
#     self.conv1 = nn.Conv2d(1, 16, 3, padding=0)         #input - 28 OUtput - 26 RF
#     self.conv2 = nn.Conv2d(16, 10, 3, padding=0)        #input - 26 OUtput - 24
#     self.pool1 = nn.MaxPool2d(2, 2)                     #input - 24 OUtput - 12
#     self.bnm2d1 = nn.BatchNorm2d(10) 
#     self.conv3 = nn.Conv2d(10, 10, 3, padding=0)        #input - 12 OUtput - 10
#     self.conv4 = nn.Conv2d(10, 16, 3, padding=0)        #input - 10 OUtput - 8
#     # self.pool2 = nn.MaxPool2d(2, 2)
#     self.bnm2d2 = nn.BatchNorm2d(16) 
#     self.conv5 = nn.Conv2d(16, 16, 3, padding=0)        #input - 8  OUtput - 6
#     self.conv6 = nn.Conv2d(16, 16, 3, padding=1)        #input - 6  OUtput - 6
#     # self.conv7 = nn.Conv2d(16, 16, 3, padding=1)      #input - 6  OUtput - 6
#     self.gap = nn.AvgPool2d(kernel_size=6)
#     self.conv8 = nn.Conv2d(16,10,1)                     

# def forward(self, x):
#     x = self.bnm2d1(self.pool1(F.relu(self.drop1(self.conv2(F.relu(self.drop1(self.conv1(x))))))))
#     x = self.bnm2d2(F.relu(self.drop1(self.conv4(F.relu(self.drop1(self.conv3(x)))))))
#     x = self.conv6(F.relu(self.drop1(self.conv5(x))))
#     # x = self.conv7(x)
#     x = self.gap(x)
#     x = self.conv8(x)
#     x = x.view(-1, 10)
#     return F.log_softmax(x)


model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 15):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.32052379846572876 batch_id=468 Accuracy=72.92: 100%|██████████| 469/469 [00:24<00:00, 19.00it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1282, Accuracy: 9627/10000 (96.27%)



loss=0.0630456879734993 batch_id=468 Accuracy=96.21: 100%|██████████| 469/469 [00:24<00:00, 19.03it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0821, Accuracy: 9750/10000 (97.50%)



loss=0.10735870152711868 batch_id=468 Accuracy=97.42: 100%|██████████| 469/469 [00:24<00:00, 19.32it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0280, Accuracy: 9900/10000 (99.00%)



loss=0.024174368008971214 batch_id=468 Accuracy=98.79: 100%|██████████| 469/469 [00:24<00:00, 18.89it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0282, Accuracy: 9914/10000 (99.14%)



loss=0.021944155916571617 batch_id=468 Accuracy=98.86: 100%|██████████| 469/469 [00:24<00:00, 19.04it/s]



Test set: Average loss: 0.0304, Accuracy: 9905/10000 (99.05%)

